# Init

In [1]:
import kuti
from kuti import applications as apps
from kuti import model_helper as mh
from kuti import image_utils
from kuti import generic
import pandas as pd, numpy as np

: 

In [ ]:
import os 
print(os.getcwd())
# os.chdir("/media/workstation/0832621B32620DCE/Ian")
print(os.getcwd())

/media/workstation/0832621B32620DCE/Ian
/media/workstation/0832621B32620DCE/Ian


In [ ]:
# ORIGINAL PATH FOR AVA-MLSP

# root_path = '/media/workstation/0832621B32620DCE/Ian/ava-mlsp/'
# ids_ava_mlsp = pd.read_csv(root_path + 'metadata/AVA_data_official_test.csv')

# ids_ava_mlsp
# input_shape = (None, None, 3)
# features_root = root_path + 'features/'
# images_path = root_path + 'images/'

# Extracting features for GET2 test

In [4]:
root_path = '/media/workstation/0832621B32620DCE/Ian/'
ids = pd.read_csv(root_path + 'mtaiq/PARA_MTAIQ_GET2_official_dataset.csv')

input_shape = (None, None, 3)
features_root = root_path + 'features_get2/'
images_path = '/media/workstation/0832621B32620DCE/PARA_Dataset/PARA/PARA_resized/'


## Choose and load pretrained model

### InceptionV3 MLSP narrow

In [ ]:
model = apps.model_inception_multigap(input_shape)
pre = apps.process_input[apps.InceptionV3]
model_name = 'iv3_mlsp_narrow'

### InceptionV3 MLSP wide

In [ ]:
model = apps.model_inception_pooled(input_shape)
pre   = apps.process_input[apps.InceptionV3]
model_name = 'iv3_mlsp_wide'

### InceptionResNetV2 MLSP narrow

In [ ]:
model = apps.model_inceptionresnet_multigap(input_shape)
pre = apps.process_input[apps.InceptionResNetV2]
model_name = 'irnv2_mlsp_narrow'

### InceptionResNetV2 MLSP wide

In [ ]:
model, pool_size = apps.model_inceptionresnet_pooled(input_shape, return_sizes=True)
pre   = apps.process_input[apps.InceptionResNetV2]
model_name = 'irnv2_mlsp_wide'

Loading InceptionResNetV2 multi-pooled with input_shape: (None, None, 3)
Creating multi-pooled model


## Save features

In [11]:
import tensorflow as tf
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

physical_devices = tf.config.experimental.list_physical_devices('GPU')
# tf.config.experimental.set_memory_growth(physical_devices[0], True)


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 10637767124019153042
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 10500833280
locality {
  bus_id: 1
  links {
  }
}
incarnation: 7782482734074538775
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1"
xla_global_id: 416903419
]


2023-01-30 17:09:17.370303: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-30 17:09:17.370710: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-30 17:09:17.370837: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-30 17:09:17.371001: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-30 17:09:17.371149: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from S

### original sized images, no augmentation

In [41]:
gen_params = dict(batch_size  = 1,
                  data_path   = images_path,
                  input_shape = ('orig',),
                  inputs = ['sessionId_imageName'],
                  outputs = [['aestheticScore'], ['qualityScore']],
                  process_fn  = pre,
                  fixed_batches = False)

helper = mh.ModelHelper(model, model_name + '_orig', ids,
                    features_root = features_root,
                    gen_params    = gen_params)

generator = helper.make_generator(ids)
# print(generator[0][0][0][0][0][0][0])

print('Saving features')
batch_size = 512
numel = len(ids)

for i in range(0,numel,batch_size):
    istop = min(i+batch_size, numel)
    # print('Processing images',i,':',istop)
    ids_batch = ids[i:istop].reset_index(drop=True)
    # print(ids_batch)
    # helper.save_activations(ids=ids_batch, verbose=True,\
    #                         groups=1,
    #                         save_as_type=np.float16)

-0.99215686
Saving features
31220


### originals, augmented with flip and crop

In [ ]:
crops = [(i,j) for i in (1,0) for j in (1,0)]
flips = (0,1)

for i, crop_pos in enumerate(crops):
    for doflip in flips:
        print('crop', crop_pos, 'flip' if doflip else '')
        process_fn = lambda im: pre(image_utils.ImageAugmenter(im, verbose=False, remap=False).\
                                    crop(0.875, crop_pos).fliplr(doflip).result)
        gen_params = dict(batch_size  = 1,
                          data_path   = images_path,
                          input_shape = ('orig',),
                          inputs = ['sessionId_imageName'],
                          outputs = [['aestheticScore'], ['qualityScore']],
                          process_fn  = process_fn,
                          fixed_batches = False)

        helper = mh.ModelHelper(model, model_name + '_fc8_orig', ids,
                            features_root = features_root,
                            gen_params    = gen_params)
        
        # generator = helper.make_generator(ids)
        # print(generator[0])

        print('Saving features')
        batch_size = 1024
        numel = len(ids)
        flag = False
        for i in range(0,numel,batch_size):
            
            istop = min(i+batch_size, numel)
            print('Processing images',i,':',istop)
            ids_batch = ids[i:istop].reset_index(drop=True)
            helper.save_activations(ids=ids_batch, verbose=True,\
                                    groups=['%g_%g_%d' %(crop_pos+(doflip,))], 
                                    save_as_type=np.float16, overwrite=False)
        #     if i == 5:
        #         flag = True
        #         break
        # if flag:
        #     break



In [11]:
# check contents of saved file
with generic.H5Helper(features_root + 'irnv2_mlsp_wide_orig/grp:1 i:1[orig] lay:final o:1[5,5,16928].h5','r') as h:
    print(h.summary())
    # print((list(h.hf.keys())))
    # print(h.hf.get('session1_iaa_pub1_.jpg'))

File size: 24.622 GB
Groups:
* session100_iaa_pub6931_.jpg/
* session100_iaa_pub6932_.jpg/
* session100_iaa_pub6933_.jpg/
* session100_iaa_pub6934_.jpg/
* session100_iaa_pub6935_.jpg/
* session100_iaa_pub6936_.jpg/
* session100_iaa_pub6937_.jpg/
* session100_iaa_pub6938_.jpg/
* session100_iaa_pub6939_.jpg/
* session100_iaa_pub6940_.jpg/
* session100_iaa_pub6941_.jpg/
* session100_iaa_pub6942_.jpg/
* session100_iaa_pub6943_.jpg/
* session100_iaa_pub6944_.jpg/
* session100_iaa_pub6945_.jpg/
* session100_iaa_pub6946_.jpg/
* session100_iaa_pub6947_.jpg/
* session100_iaa_pub6948_.jpg/
* session100_iaa_pub6949_.jpg/
* session100_iaa_pub6950_.jpg/
* session100_iaa_pub6951_.jpg/
* session100_iaa_pub6952_.jpg/
* session100_iaa_pub6953_.jpg/
* session100_iaa_pub6954_.jpg/
* session100_iaa_pub6955_.jpg/
* session100_iaa_pub6956_.jpg/
* session100_iaa_pub6957_.jpg/
* session100_iaa_pub6958_.jpg/
* session100_iaa_pub6959_.jpg/
* session100_iaa_pub6960_.jpg/
* session100_iaa_pub6961_.jpg/
* session1

# Extracting features for GET3 test

In [2]:
root_path = '/media/workstation/0832621B32620DCE/Ian/'
ids = pd.read_csv(root_path + 'mtaiq/PARA_MTAIQ_GET3_official_dataset.csv')

input_shape = (None, None, 3)
features_root = root_path + 'features_get3/'
images_path = '/media/workstation/0832621B32620DCE/PARA_Dataset/PARA/PARA_resized/'


## Choose and load pretrained model

### InceptionResNetV2 MLSP wide

In [3]:
model = apps.model_inceptionresnet_pooled(input_shape)
pre   = apps.process_input[apps.InceptionResNetV2]
model_name = 'irnv2_mlsp_wide'

Loading InceptionResNetV2 multi-pooled with input_shape: (None, None, 3)


2023-01-26 02:04:25.115863: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-26 02:04:25.144789: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-26 02:04:25.145024: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-26 02:04:25.145587: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropri

Creating multi-pooled model


## Save features

In [4]:
import tensorflow as tf
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

physical_devices = tf.config.experimental.list_physical_devices('GPU')
# tf.config.experimental.set_memory_growth(physical_devices[0], True)


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 15703890552849797949
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 10386800640
locality {
  bus_id: 1
  links {
  }
}
incarnation: 4458143984700764472
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1"
xla_global_id: 416903419
]


2023-01-26 02:04:34.849796: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-26 02:04:34.850029: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-26 02:04:34.850153: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-26 02:04:34.850319: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-26 02:04:34.850467: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from S

### original sized images, no augmentation

In [6]:
gen_params = dict(batch_size  = 1,
                  data_path   = images_path,
                  input_shape = ('orig',),
                  inputs = ['sessionId_imageName'],
                  outputs = [['aestheticScore'], ['qualityScore']],
                  process_fn  = pre,
                  fixed_batches = False)

helper = mh.ModelHelper(model, model_name + '_orig', ids,
                    features_root = features_root,
                    gen_params    = gen_params)

generator = helper.make_generator(ids)
print(generator[0])

print('Saving features')
batch_size = 512
numel = len(ids)
print(numel)

for i in range(0,numel,batch_size):
    istop = min(i+batch_size, numel)
    print('Processing images',i,':',istop)
    ids_batch = ids[i:istop].reset_index(drop=True)
    # print(ids_batch)
    # helper.save_activations(ids=ids_batch, verbose=True,\
    #                         groups=1,
    #                         save_as_type=np.float16)

Saving features
31220
Processing images 0 : 512
Generating batches |==================================================| 100% 
Writing datasets |==================================================| 100% 
Processing images 512 : 1024
Generating batches |==================================================| 100% 
Writing datasets |==================================================| 100% 
Processing images 1024 : 1536
Generating batches |==================================================| 100% 
Writing datasets |==================================================| 100% 
Processing images 1536 : 2048
Generating batches |==================================================| 100% 
Writing datasets |==================================================| 100% 
Processing images 2048 : 2560
Generating batches |==================================================| 100% 
Writing datasets |==================================================| 100% 
Processing images 2560 : 3072
Generating batches |=============

In [7]:
# check contents of saved file
with generic.H5Helper(features_root + 'irnv2_mlsp_wide_orig/grp:1 i:1[orig] lay:final o:1[5,5,16928].h5','r') as h:
    print(h.summary())
    # print((list(h.hf.keys())))
    # print(h.hf.get('session1_iaa_pub1_.jpg'))

File size: 24.622 GB
Groups:
* session100_iaa_pub6931_.jpg/
* session100_iaa_pub6932_.jpg/
* session100_iaa_pub6933_.jpg/
* session100_iaa_pub6934_.jpg/
* session100_iaa_pub6935_.jpg/
* session100_iaa_pub6936_.jpg/
* session100_iaa_pub6937_.jpg/
* session100_iaa_pub6938_.jpg/
* session100_iaa_pub6939_.jpg/
* session100_iaa_pub6940_.jpg/
* session100_iaa_pub6941_.jpg/
* session100_iaa_pub6942_.jpg/
* session100_iaa_pub6943_.jpg/
* session100_iaa_pub6944_.jpg/
* session100_iaa_pub6945_.jpg/
* session100_iaa_pub6946_.jpg/
* session100_iaa_pub6947_.jpg/
* session100_iaa_pub6948_.jpg/
* session100_iaa_pub6949_.jpg/
* session100_iaa_pub6950_.jpg/
* session100_iaa_pub6951_.jpg/
* session100_iaa_pub6952_.jpg/
* session100_iaa_pub6953_.jpg/
* session100_iaa_pub6954_.jpg/
* session100_iaa_pub6955_.jpg/
* session100_iaa_pub6956_.jpg/
* session100_iaa_pub6957_.jpg/
* session100_iaa_pub6958_.jpg/
* session100_iaa_pub6959_.jpg/
* session100_iaa_pub6960_.jpg/
* session100_iaa_pub6961_.jpg/
* session1

# Features get all Users

In [3]:
root_path = '/media/workstation/0832621B32620DCE/Ian/'
ids = pd.read_csv(root_path + 'mtaiq/PARA_MTAIQ_All_User_official_dataset.csv')

input_shape = (None, None, 3)
features_root = root_path + 'features_all_user/'
images_path = '/media/workstation/0832621B32620DCE/PARA_Dataset/PARA/PARA_resized/'


## Choose and load pre-trained model

### InceptionResNetV2 MLSP Wide

In [4]:
model = apps.model_inceptionresnet_pooled(input_shape)
pre   = apps.process_input[apps.InceptionResNetV2]
model_name = 'irnv2_mlsp_wide'

Loading InceptionResNetV2 multi-pooled with input_shape: (None, None, 3)


2023-02-04 15:58:56.013244: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-04 15:58:56.677059: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-04 15:58:56.677256: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-04 15:58:56.697106: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropri

Creating multi-pooled model


### Save Features

### original sized images, no augmentation

In [5]:
gen_params = dict(batch_size  = 1,
                  data_path   = images_path,
                  input_shape = ('orig',),
                  inputs = ['sessionId_imageName'],
                  outputs = [['aestheticScore'], ['qualityScore']],
                  process_fn  = pre,
                  fixed_batches = False)

helper = mh.ModelHelper(model, model_name + '_orig', ids,
                    features_root = features_root,
                    gen_params    = gen_params)

# generator = helper.make_generator(ids)
# print(generator[0])

print('Saving features')
batch_size = 512
numel = len(ids)
print(numel)

for i in range(0,numel,batch_size):
    istop = min(i+batch_size, numel)
    print('Processing images',i,':',istop)
    ids_batch = ids[i:istop].reset_index(drop=True)
    # print(ids_batch)
    helper.save_activations(ids=ids_batch, verbose=True,\
                            groups=1,
                            save_as_type=np.float16)

Saving features
31220
Processing images 0 : 512


2023-02-04 15:59:44.369028: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8201
2023-02-04 15:59:51.392125: W tensorflow/stream_executor/gpu/asm_compiler.cc:111] *** WARNING *** You are using ptxas 10.0.145, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.


Generating batches |==================================================| 100% 
Writing datasets |==================================================| 100% 
Processing images 512 : 1024
Generating batches |==================================================| 100% 
Writing datasets |==================================================| 100% 
Processing images 1024 : 1536
Generating batches |==================================================| 100% 
Writing datasets |==================================================| 100% 
Processing images 1536 : 2048
Generating batches |==================================================| 100% 
Writing datasets |==================================================| 100% 
Processing images 2048 : 2560
Generating batches |==================================================| 100% 
Writing datasets |==================================================| 100% 
Processing images 2560 : 3072
Generating batches |==================================================| 100% 
Wri

In [6]:
# check contents of saved file
with generic.H5Helper(features_root + 'irnv2_mlsp_wide_orig/grp:1 i:1[orig] lay:final o:1[5,5,16928].h5','r') as h:
    print(h.summary())
    # print((list(h.hf.keys())))
    # print(h.hf.get('session1_iaa_pub1_.jpg'))

File size: 24.622 GB
Groups:
* session100_iaa_pub6931_.jpg/
* session100_iaa_pub6932_.jpg/
* session100_iaa_pub6933_.jpg/
* session100_iaa_pub6934_.jpg/
* session100_iaa_pub6935_.jpg/
* session100_iaa_pub6936_.jpg/
* session100_iaa_pub6937_.jpg/
* session100_iaa_pub6938_.jpg/
* session100_iaa_pub6939_.jpg/
* session100_iaa_pub6940_.jpg/
* session100_iaa_pub6941_.jpg/
* session100_iaa_pub6942_.jpg/
* session100_iaa_pub6943_.jpg/
* session100_iaa_pub6944_.jpg/
* session100_iaa_pub6945_.jpg/
* session100_iaa_pub6946_.jpg/
* session100_iaa_pub6947_.jpg/
* session100_iaa_pub6948_.jpg/
* session100_iaa_pub6949_.jpg/
* session100_iaa_pub6950_.jpg/
* session100_iaa_pub6951_.jpg/
* session100_iaa_pub6952_.jpg/
* session100_iaa_pub6953_.jpg/
* session100_iaa_pub6954_.jpg/
* session100_iaa_pub6955_.jpg/
* session100_iaa_pub6956_.jpg/
* session100_iaa_pub6957_.jpg/
* session100_iaa_pub6958_.jpg/
* session100_iaa_pub6959_.jpg/
* session100_iaa_pub6960_.jpg/
* session100_iaa_pub6961_.jpg/
* session1